In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
import copy
import socket
from tqdm import tqdm
import torch
import pickle
from torch import optim
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "../../")))
from libs import fl, nn, agg, data, poison, log, sim

In [2]:
# Save Logs To File (info | debug | warning | error | critical) [optional]
log.init("debug")
#log.init("info", "federated.log")
#log.init("debug", "flkafka.log")

In [3]:
class FedArgs():
    def __init__(self):
        self.num_clients = 50
        self.epochs = 10
        self.local_rounds = 1
        self.client_batch_size = 32
        self.test_batch_size = 128
        self.learning_rate = 1e-4
        self.weight_decay = 1e-5
        self.cuda = True
        self.seed = 1
        self.tb = SummaryWriter('../../out/runs/federated/FLTrust', comment="Mnist Centralized Federated training")

fedargs = FedArgs()

In [9]:
use_cuda = fedargs.cuda and torch.cuda.is_available()
torch.manual_seed(fedargs.seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {"num_workers": 1, "pin_memory": True} if use_cuda else {}

In [10]:
print(torch.cuda.is_available())

False


In [6]:
host = socket.gethostname()
clients = [host + "(" + str(client + 1) + ")" for client in range(fedargs.num_clients)]

In [7]:
#Initialize Global and Client models
global_model = nn.ModelMNIST()
client_models = {client: copy.deepcopy(global_model) for client in clients}

# Function for training
def train_model(_model, train_loader, fedargs, device):
    model, loss = fl.client_update(_model,
                                train_loader,
                                fedargs.learning_rate,
                                fedargs.weight_decay,
                                fedargs.local_rounds,
                                device)
    model_update = agg.sub_model(_model, model)
    return model_update, model, loss

In [8]:
# Load MNIST Data to clients
train_data, test_data = data.load_dataset("mnist")

In [9]:
# For FLTrust
#############Skip this section for running other averaging
FLTrust = True
root_ratio = 0.01
train_data, root_data = torch.utils.data.random_split(train_data, [int(len(train_data) * (1-root_ratio)), 
                                                              int(len(train_data) * root_ratio)])
root_loader = torch.utils.data.DataLoader(root_data, batch_size=fedargs.client_batch_size, shuffle=True, **kwargs)

#global_model, _ = train_model(global_model, root_loader, fedargs, device)
#client_models = {client: copy.deepcopy(global_model) for client in clients}
#############

In [10]:
clients_data = data.split_data(train_data, clients)

In [11]:
# Poison a client
################Skip this section for running without poison
for client in range(10):
    clients_data[clients[client]] = poison.label_flip(clients_data[clients[client]], 4, 9, poison_percent = -1)
    
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 6, 2, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 3, 8, poison_percent = 1)
#clients_data[clients[0]] = poison.label_flip(clients_data[clients[0]], 1, 5, poison_percent = 1)

In [12]:
client_train_loaders, _ = data.load_client_data(clients_data, fedargs.client_batch_size, None, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=fedargs.test_batch_size, shuffle=True, **kwargs)

clients_info = {
        client: {"train_loader": client_train_loaders[client]}
        for client in clients
    }

In [13]:
B1 = copy.deepcopy(global_model)

C1 = copy.deepcopy(global_model)
C2 = copy.deepcopy(global_model)

C3 = copy.deepcopy(global_model)
C4 = copy.deepcopy(global_model)
C5 = copy.deepcopy(global_model)

#for i in range(3):
_B1, B1, _ = train_model(B1, root_loader, fedargs, device)

_C3, C3, _ = train_model(C3, clients_info[list(clients_info.keys())[21]]['train_loader'], fedargs, device)
_C4, C4, _ = train_model(C4, clients_info[list(clients_info.keys())[22]]['train_loader'], fedargs, device)
_C5, C5, _ = train_model(C5, clients_info[list(clients_info.keys())[23]]['train_loader'], fedargs, device)

In [14]:
#for j in range(20):
_C1, C1, _ = train_model(C1, clients_info[list(clients_info.keys())[1]]['train_loader'], fedargs, device)
_C2, C2, _ = train_model(C2, clients_info[list(clients_info.keys())[2]]['train_loader'], fedargs, device)

print(sim.grad_cosine_similarity(_B1, _C1))
print(sim._grad_cosine_similarity(_B1, _C1))

print("###################################")

print(sim.grad_cosine_similarity(_B1, _C2))
print(sim._grad_cosine_similarity(_B1, _C2))

print("###################################")

print(sim.grad_cosine_similarity(_B1, _C3))
print(sim._grad_cosine_similarity(_B1, _C3))

print("###################################")

print(sim.grad_cosine_similarity(_B1, _C4))
print(sim._grad_cosine_similarity(_B1, _C4))

print("###################################")

print(sim.grad_cosine_similarity(_B1, _C5))
print(sim._grad_cosine_similarity(_B1, _C5))

/home2/harsh_1921cs01/hub/F3IA/fl/libs/sim.py:45: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  _param_list = nd.array(param_list).squeeze()


0.10440999
[array(0.24124, dtype=float32), array(0.0403336, dtype=float32), array(0.10411313, dtype=float32), array(0.19308683, dtype=float32)]
0.14469338953495026
###################################
0.087574705
[array(0.07573985, dtype=float32), array(0.03305238, dtype=float32), array(0.09019455, dtype=float32), array(0.20181827, dtype=float32)]
0.10020126402378082
###################################
0.52985364
[array(0.56401443, dtype=float32), array(0.5321269, dtype=float32), array(0.5289359, dtype=float32), array(0.5586356, dtype=float32)]
0.5459282100200653
###################################
0.56827354
[array(0.5533229, dtype=float32), array(0.51556665, dtype=float32), array(0.5624943, dtype=float32), array(0.5380424, dtype=float32)]
0.5423565655946732
###################################
0.5390002
[array(0.5482059, dtype=float32), array(0.5267581, dtype=float32), array(0.5415623, dtype=float32), array(0.51956785, dtype=float32)]
0.5340235382318497


In [163]:
def sub_model(model1, model2):
    params1 = model1.state_dict().copy()
    params2 = model2.state_dict().copy()
    
    params1['conv1.weight'] = params2['conv1.weight']
    #params1['fc1.weight'] = params2['fc1.weight']
    
    model = copy.deepcopy(model1)
    model.load_state_dict(params1, strict=False)
    return model

_C1_ = copy.deepcopy(_C1)
_C1_ = sub_model(_B1, _C1_)

_C2_ = copy.deepcopy(_C2)
_C2_ = sub_model(_B1, _C2_)

#print(sim.grad_cosine_similarity(_B1, _C1_))
#print(sim._grad_cosine_similarity(_B1, _C1_))

avgargs = {"base_update": _B1}
t1 = fl.federated_avg({'a': _C1, 'b': _C2, 'c': _C3, 'd': _C4, 'e': _C5}, B1, agg.Rule.FLTrust, **avgargs)
t2 = fl.federated_avg({'a': _C1_, 'b': _C2_, 'c': _C3, 'd': _C4, 'e': _C5}, B1, agg.Rule.FLTrust, **avgargs)

print(fl.eval(t1, test_loader, device))
print(fl.eval(t2, test_loader, device))

2021-08-24 09:48:56,834 - /home2/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:90) : FLTrust Score [0.104, 0.088, 0.53, 0.568, 0.539] [MainProcess : MainThread (INFO)]
2021-08-24 09:48:57,422 - /home2/harsh_1921cs01/hub/F3IA/fl/libs/agg.py::FLTrust(l:90) : FLTrust Score [1.001, 1.001, 0.53, 0.568, 0.539] [MainProcess : MainThread (INFO)]


{'test_loss': 1.5952611003875732, 'correct': 6643, 'accuracy': 66.43}
{'test_loss': 1.6413420360565185, 'correct': 5800, 'accuracy': 57.99999999999999}


In [15]:
from mxnet import nd as mnd

fb1, bslist = sim.get_net_arr(_B1)
fc1, cslist = sim.get_net_arr(_C1)
fc5, cslist = sim.get_net_arr(_C5)

print(bslist)

print(fb1.shape)

cs1 = mnd.dot(mnd.array(fb1), mnd.array(fc1)) / (mnd.norm(mnd.array(fb1)) + 1e-9) / (mnd.norm(mnd.array(fc1)) + 1e-9)
cs5 = mnd.dot(mnd.array(fb1), mnd.array(fc5)) / (mnd.norm(mnd.array(fb1)) + 1e-9) / (mnd.norm(mnd.array(fc5)) + 1e-9)

print(cs1, cs5)
#_T1 = sim.get_arr_net(_C1, fc1, slist)

[(32, 1, 3, 3), (32,), (64, 32, 3, 3), (64,), (128, 9216), (128,), (10, 128), (10,)]
(1199882,)

[0.10440999]
<NDArray 1 @cpu(0)> 
[0.5390002]
<NDArray 1 @cpu(0)>


In [ ]:
model=nn.denoising_model().to(device)
criterion=torch.nn.MSELoss()
optimizer=optim.SGD(model.parameters(),lr=0.5,weight_decay=1e-5)

output = fc1
model.train()
for epcoh in range(10):
    optimizer.zero_grad()
    output = model(torch.from_numpy(fc1))
    cs1 = mnd.dot(mnd.array(fb1), mnd.array(output.detach().numpy())) / (mnd.norm(mnd.array(fb1)) + 1e-9) / (mnd.norm(mnd.array(output.detach().numpy())) + 1e-9)
    _loss = criterion(output, torch.from_numpy(fb1))
    _loss = _loss + cs1.asnumpy()[0]
    print(_loss)
    _loss.backward()
    optimizer.step()